# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
import findspark

In [2]:
findspark.init('/home/vlad1974/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [5]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [6]:
data.show(3)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
+----+--------------------+
only showing top 3 rows



In [7]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [8]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length',length(data['text']))

In [11]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [12]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [14]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [16]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

In [17]:
from pyspark.ml.classification import NaiveBayes

In [18]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [19]:
from pyspark.ml import Pipeline

In [20]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [21]:
cleaner = data_prep_pipe.fit(data)

In [22]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [23]:
clean_data = clean_data.select(['label','features'])

In [24]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
+-----+--------------------+
only showing top 5 rows



In [25]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [26]:
spam_predictor = nb.fit(training)

In [27]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [28]:
test_results = spam_predictor.transform(testing)

In [29]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,2,14,...|[-605.10691067191...|[1.0,7.2864010316...|       0.0|
|  0.0|(13459,[0,1,3,10,...|[-603.94938330022...|[1.0,1.9741759317...|       0.0|
|  0.0|(13459,[0,1,5,15,...|[-746.16249723156...|[1.0,3.9506186564...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-880.70371279089...|[1.0,1.2565652618...|       0.0|
|  0.0|(13459,[0,1,12,34...|[-888.37443861682...|[1.0,7.1255196209...|       0.0|
|  0.0|(13459,[0,1,15,19...|[-1367.5823848564...|[1.0,1.6225756752...|       0.0|
|  0.0|(13459,[0,1,16,21...|[-668.18293015404...|[1.0,2.1383526570...|       0.0|
|  0.0|(13459,[0,1,21,29...|[-967.30042376351...|[1.0,2.1387543877...|       0.0|
|  0.0|(13459,[0,1,25,66...|[-1340.1019315081...|[1.0,4.3406784267...|       0.0|
|  0.0|(13459,[0

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9298996714899107


## Accuracy of model : 0.93